In [543]:
#If you need to install in modules in jupyter notebook 
%pip install scikit-learn
%pip install pandas
%pip install matplotlib 
%pip install seaborn 
%pip install graphviz
%pip install scikit-plot   
%pip install statsmodels   


#Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
import datetime # Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# Common imports
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats
import scikitplot as skplt
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.linear_model import LinearRegression
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# to make this notebook's output stable across runs
np.random.seed(42)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

os.makedirs(IMAGES_PATH, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):

    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [544]:
#changes the output for the print statements
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',0)

In [545]:
#Pull 
df = pd.read_csv('https://raw.githubusercontent.com/Rodrig79/Machine-Learning-Data-Analysis-Project/master/rawData/pokemon.csv') 

In [546]:
from sklearn import preprocessing

#Data Cleaning
df = df[~df.Name.str.contains('Mega')] #removed pokemon with "Mega" in it
df = df.drop(columns = ["Name","Type 1","Type 2","#","Generation"]) #Removed columns with names

#normalization
# X = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(X)
# df=pd.DataFrame(x_scaled, columns=df.columns)


In [547]:
 #Splitting dataset to x and y
# y = df.Attack
# X = df[['Defense','HP','Sp. Atk','Sp. Def','Speed']]
#seperating data
df = df.dropna() 
y = df['Attack']
X = df[['Defense','HP','Speed']] 
X.dropna()
y.dropna()

0      49 
1      62 
2      82 
4      52 
5      64 
       .. 
794    100
795    100
797    110
798    160
799    110
Name: Attack, Length: 751, dtype: int64

In [548]:
X

,Defense,HP,Speed
0,49,45,45
1,63,60,60
2,83,80,80
4,43,39,65
5,58,58,80
...,...,...,...
794,121,108,95
795,150,50,50
797,60,80,70
798,60,80,80


# MultiVariate  Regression with sklearn

In [549]:
 from sklearn.linear_model import LinearRegression
 from sklearn import metrics

In [550]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

linreg = LinearRegression().fit(X_train, y_train)

print('Wheat dataset')
print('linear model intercept: {}'
     .format(linreg.intercept_))
print('linear model coeff:\n{}'
     .format(linreg.coef_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))


# make predictions on the testing set
y_pred = linreg.predict(X_test)
print('RMSE: {:.3f}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

Wheat dataset
linear model intercept: 5.305692824017925
linear model coeff:
[0.33084547 0.39032057 0.29685746]
R-squared score (training): 0.370
R-squared score (test): 0.374
RMSE: 23.266


# Linear Regression with Gradient Descent 

In [551]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [552]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline
# excel_file = 'Folds5x2_pp.xlsx'
# data = pd.read_excel(excel_file)
# print(data.head(5))

In [553]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [683]:
def cost_function(X, Y, B):
    m = len(Y)
    J = np.sum((X.dot(B) - Y) ** 2)/(2 * m)
    return J

def batch_gradient_descent(X, Y, B, alpha, iterations):
    cost_history = [0] * iterations
    m = len(Y)

    for iteration in range(iterations):
        #print(iteration)
        # Hypothesis Values
        h = X.dot(B)
        # Difference b/w Hypothesis and Actual Y
        loss = h - Y
        # Gradient Calculation
        gradient = X.T.dot(loss) / m
        # Changing Values of B using Gradient
        B = B - alpha * gradient
        # New Cost Value
        cost = cost_function(X, Y, B)
        cost_history[iteration] = cost 
    return B, cost_history 
def pred(x_test, newB):
    return x_test.dot(newB)

In [684]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 42)  

In [699]:
# Initial Coefficients
B = np.zeros(X_train.shape[1])
alpha = .02
iter_ = 5000
newB, cost_history = batch_gradient_descent(X_train, y_train, B, alpha, iter_)


In [700]:
y_ = pred(X_test,newB)

In [703]:
def r2(y_,y):
    sst = np.sum((y-y.mean())**2)
    ssr = np.sum((y_-y)**2)
    r2 = 1-(ssr/sst)
    return(r2)
#----------------
# r2 = abs(r2(y_,y_test))
r2 = r2(y_,y_test)
print("r2:",r2)

r2: -7.466936664319855


In [702]:
ans_ = abs(pred(X_test[3],newB))
ans_ = pred(X_test[3],newB)
ans_

-23.170235462156274

In [690]:

cost_history

[3325.8717355671743,
 3325.626575677521,
 3325.3817490067427,
 3325.137255101492,
 3324.893093509043,
 3324.6492637772844,
 3324.40576545472,
 3324.1625980904687,
 3323.919761234263,
 3323.6772544364485,
 3323.4350772479834,
 3323.1932292204347,
 3322.951709905982,
 3322.7105188574155,
 3322.4696556281306,
 3322.229119772134,
 3321.9889108440375,
 3321.7490283990605,
 3321.5094719930266,
 3321.270241182365,
 3321.031335524108,
 3320.7927545758907,
 3320.554497895952,
 3320.3165650431292,
 3320.078955576863,
 3319.8416690571917,
 3319.604705044754,
 3319.3680631007865,
 3319.1317427871213,
 3318.8957436661885,
 3318.660065301015,
 3318.4247072552193,
 3318.189669093016,
 3317.9549503792123,
 3317.7205506792084,
 3317.486469558996,
 3317.252706585156,
 3317.019261324862,
 3316.786133345874,
 3316.5533222165413,
 3316.3208275058028,
 3316.088648783181,
 3315.856785618786,
 3315.625237583313,
 3315.394004248041,
 3315.1630851848336,
 3314.9324799661363,
 3314.7021881649766,
 3314.472209354

# Regularized Regression Model
## Ridge

In [559]:
from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

linridge = Ridge(alpha=25.0).fit(X_train, y_train)

print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

ridge regression linear model intercept: 75.61029319590861
ridge regression linear model coeff:
[9.45954091 9.77904476 7.9729408 ]
R-squared score (training): 0.369
R-squared score (test): 0.375
Number of non-zero features: 3


In [560]:
# Ridge regression with feature normalization

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linridge = Ridge(alpha=20.0).fit(X_train_scaled, y_train)

print('Wheat dataset')
print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

Wheat dataset
ridge regression linear model intercept: 49.33056850867053
ridge regression linear model coeff:
[30.092738   25.32929225 25.27089312]
R-squared score (training): 0.231
R-squared score (test): 0.247
Number of non-zero features: 3


In [561]:
# Ridge regression with regularization parameter: alpha

print('Ridge regression: effect of alpha regularization parameter\n')
for this_alpha in [0, 1, 10, 20, 50, 100, 1000]:
    linridge = Ridge(alpha = this_alpha).fit(X_train_scaled, y_train)
    r2_train = linridge.score(X_train_scaled, y_train)
    r2_test = linridge.score(X_test_scaled, y_test)
    num_coeff_bigger = np.sum(abs(linridge.coef_) > 1.0)
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
r-squared training: {:.2f}, r-squared test: {:.2f}\n'
         .format(this_alpha, num_coeff_bigger, r2_train, r2_test))
  

Ridge regression: effect of alpha regularization parameter

Alpha = 0.00
num abs(coeff) > 1.0: 3, r-squared training: 0.37, r-squared test: 0.37

Alpha = 1.00
num abs(coeff) > 1.0: 3, r-squared training: 0.37, r-squared test: 0.38

Alpha = 10.00
num abs(coeff) > 1.0: 3, r-squared training: 0.29, r-squared test: 0.32

Alpha = 20.00
num abs(coeff) > 1.0: 3, r-squared training: 0.23, r-squared test: 0.25

Alpha = 50.00
num abs(coeff) > 1.0: 3, r-squared training: 0.14, r-squared test: 0.14

Alpha = 100.00
num abs(coeff) > 1.0: 3, r-squared training: 0.08, r-squared test: 0.08

Alpha = 1000.00
num abs(coeff) > 1.0: 1, r-squared training: 0.01, r-squared test: 0.00



## Lasso Regression

In [562]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linlasso = Lasso(alpha=1, max_iter = 10000).fit(X_train_scaled, y_train)

print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('Non-zero features: {}'
     .format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'
     .format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'
     .format(linlasso.score(X_test_scaled, y_test)))
print('Features with non-zero weight (sorted by absolute magnitude):')

for e in sorted (list(zip(list(X), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

lasso regression linear model intercept: 51.691515991159186
lasso regression linear model coeff:
[34.93234036 13.83541656 22.61395754]
Non-zero features: 3
R-squared score (training): 0.212
R-squared score (test): 0.228

Features with non-zero weight (sorted by absolute magnitude):
	[-0.76812684 -0.91287325 -0.77648137], 34.932
	[0.38012857 0.43697336 0.48128734], 22.614
	[-0.29531579 -0.33436756 -0.23743764], 13.835


In [564]:

print('Lasso regression: effect of alpha regularization\n\
parameter on number of features kept in final model\n')

for alpha in [0.5, 1, 2, 3, 5, 10, 20, 50]:
    linlasso = Lasso(alpha, max_iter = 10000).fit(X_train_scaled, y_train)
    r2_train = linlasso.score(X_train_scaled, y_train)
    r2_test = linlasso.score(X_test_scaled, y_test)
    
    print('Alpha = {:.2f}\nFeatures kept: {}, r-squared training: {:.2f}, \
r-squared test: {:.2f}\n'
         .format(alpha, np.sum(linlasso.coef_ != 0), r2_train, r2_test))

Lasso regression: effect of alpha regularization
parameter on number of features kept in final model

Alpha = 0.50
Features kept: 3, r-squared training: 0.33, r-squared test: 0.35

Alpha = 1.00
Features kept: 3, r-squared training: 0.21, r-squared test: 0.23

Alpha = 2.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.01

Alpha = 3.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.01

Alpha = 5.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.01

Alpha = 10.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.01

Alpha = 20.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.01

Alpha = 50.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.01



## Polynomial regression

In [565]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold # import k-fold validation
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
    

In [566]:
for degree in [2, 3, 5,10]:
    poly = PolynomialFeatures(degree)
    X_F1_poly = poly.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y, random_state=0)
    linreg = LinearRegression().fit(X_train, y_train)

    print('POLYNOMIAL DEGREE-{} W/ LINEAR REGRESSION'.format(degree))
    print('(poly deg {}) linear model coeff (w):\n{}'
         .format(degree, linreg.coef_))
    print('(poly deg {}) linear model intercept (b): {:.3f}'
         .format(degree, linreg.intercept_))
    print('(poly deg {}) R-squared score (training): {:.3f}'
         .format(degree, linreg.score(X_train, y_train)))
    print('(poly deg {}) R-squared score (test): {:.3f}'
         .format(degree, linreg.score(X_test, y_test)))
    print('(poly deg {}) RMSE score (train): {:.3f}'
         .format(degree, sqrt(mean_squared_error(y_train, linreg.predict(X_train)))))
    print('(poly deg {}) RMSE score (test): {:.3f}\n'
         .format(degree, sqrt(mean_squared_error(y_test, linreg.predict(X_test)))))

plot_learning_curves(linreg,  X,y)

POLYNOMIAL DEGREE-2 W/ LINEAR REGRESSION
(poly deg 2) linear model coeff (w):
[ 0.          9.27494063 14.88858744  6.2633744  -1.71614443  3.46508052
 -2.36960433 -2.0656223   2.2869867  -0.0486266 ]
(poly deg 2) linear model intercept (b): 78.221
(poly deg 2) R-squared score (training): 0.446
(poly deg 2) R-squared score (test): 0.468
(poly deg 2) RMSE score (train): 22.361
(poly deg 2) RMSE score (test): 21.449

POLYNOMIAL DEGREE-3 W/ LINEAR REGRESSION
(poly deg 3) linear model coeff (w):
[ 0.         10.04600121 16.51410576  3.8567487  -1.95735901  4.63335918
 -3.56010278 -2.93175914  3.60510299 -0.61092097  0.31269739  0.67298507
  1.71459649 -1.15030386  0.82323137 -0.75006686 -0.24894993 -0.50930664
 -0.50459191  0.37132156]
(poly deg 3) linear model intercept (b): 79.322
(poly deg 3) R-squared score (training): 0.460
(poly deg 3) R-squared score (test): 0.422
(poly deg 3) RMSE score (train): 22.073
(poly deg 3) RMSE score (test): 22.357

POLYNOMIAL DEGREE-5 W/ LINEAR REGRESSION

NameError: name 'plot_learning_curves' is not defined

In [567]:
for degree in [2, 3, 5, 10]:
    poly = PolynomialFeatures(degree)
    X_F1_poly = poly.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y, random_state=0)
    linreg = Ridge().fit(X_train, y_train)

    print('POLYNOMIAL DEGREE-{} W/ RIDGE REGRESSION'.format(degree))
    print('(poly deg {}) linear model coeff (w):\n{}'
         .format(degree, linreg.coef_))
    print('(poly deg {}) linear model intercept (b): {:.3f}'
         .format(degree, linreg.intercept_))
    print('(poly deg {}) R-squared score (training): {:.3f}'
         .format(degree, linreg.score(X_train, y_train)))
    print('(poly deg {}) R-squared score (test): {:.3f}'
         .format(degree, linreg.score(X_test, y_test)))
    print('(poly deg {}) RMSE score (train): {:.3f}'
         .format(degree, sqrt(mean_squared_error(y_train, linreg.predict(X_train)))))
    print('(poly deg {}) RMSE score (test): {:.3f}\n'
         .format(degree, sqrt(mean_squared_error(y_test, linreg.predict(X_test)))))


POLYNOMIAL DEGREE-2 W/ RIDGE REGRESSION
(poly deg 2) linear model coeff (w):
[ 0.          9.26871585 14.85093921  6.26015403 -1.71335056  3.45958078
 -2.36101212 -2.06108392  2.2770491  -0.04612315]
(poly deg 2) linear model intercept (b): 78.213
(poly deg 2) R-squared score (training): 0.446
(poly deg 2) R-squared score (test): 0.468
(poly deg 2) RMSE score (train): 22.361
(poly deg 2) RMSE score (test): 21.445

POLYNOMIAL DEGREE-3 W/ RIDGE REGRESSION
(poly deg 3) linear model coeff (w):
[ 0.         10.00261414 16.38346877  3.84937049 -1.950071    4.57036623
 -3.55003185 -2.90750989  3.57237401 -0.60189014  0.31286385  0.70291667
  1.71901529 -1.11820182  0.79729862 -0.7417496  -0.24382326 -0.48711814
 -0.47377904  0.37102814]
(poly deg 3) linear model intercept (b): 79.311
(poly deg 3) R-squared score (training): 0.460
(poly deg 3) R-squared score (test): 0.420
(poly deg 3) RMSE score (train): 22.074
(poly deg 3) RMSE score (test): 22.384

POLYNOMIAL DEGREE-5 W/ RIDGE REGRESSION
(p

/usr/local/Cellar/jupyterlab/2.2.5/libexec/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.85983e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
